In [43]:
import os

In [44]:
%pwd

'd:\\Projects\\ChickenDisease'

In [45]:
os.chdir("D:\Projects\ChickenDisease\Chicken-Disease")

In [46]:
%pwd

'D:\\Projects\\ChickenDisease\\Chicken-Disease'

In [64]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path

In [65]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [66]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

In [67]:
import os
import urllib.request  as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [78]:
class PrepareCallback:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath= str(self.config.checkpoint_model_filepath),
            save_best_only=True)
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [79]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callbacks_list= prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2023-09-05 16:25:53,613:INFO: common:yaml file : config\config.yaml loaded successfully]
[2023-09-05 16:25:53,619:INFO: common:yaml file : params.yaml loaded successfully]
[2023-09-05 16:25:53,621:INFO: common:Directory at : artifacts created successfully]
[2023-09-05 16:25:53,623:INFO: common:Directory at : artifacts\prepare_callbacks\checkpoint_dir created successfully]
[2023-09-05 16:25:53,625:INFO: common:Directory at : artifacts\prepare_callbacks\tensorboard_log_dir created successfully]
